In [ ]:
import torch.nn as nn
import torch 

class SelfAttention_v1(nn.Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()      # Khởi tạo lớp cha nn.Module
        self.dim_out = dim_out   # Kích thước đầu ra
        # rand - phân phối đều liên tục trong khoảng [0, 1)
        self.W_query = nn.Parameter(torch.rand(dim_in, dim_out))  # Ma trận trọng số cho query
        self.W_key = nn.Parameter(torch.rand(dim_in, dim_out))    # Ma trận trọng số cho key
        self.W_value = nn.Parameter(torch.rand(dim_in, dim_out))  # Ma trận trọng số cho value

    def forward(self, x):
        queries = x @ self.W_query # Tính queries
        keys = x @ self.W_key      # Tính keys
        values = x @ self.W_value  # Tính values

        attn_scores = queries @ keys.T      # Tính attention scores (omega)
        attn_weights = torch.softmax(
            attn_scores / self.W_key.shape[-1]**0.5, dim=-1
        )

        context_vectors = attn_weights @ values  # Tính context vectors (z)

        return context_vectors

In [4]:
inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your (x^1)
    [0.55, 0.87, 0.66], # journey (x^2)
    [0.57, 0.85, 0.64], # starts (x^3)
    [0.22, 0.58, 0.33], # with (x^4)
    [0.77, 0.25, 0.10], # one (x^5)
    [0.05, 0.80, 0.55]] # step (x^6)
)

In [ ]:
torch.manual_seed(123)
self_attention_v1 = SelfAttention_v1(dim_in=3, dim_out=2)   
print(self_attention_v1(inputs))    # run forward with inputs

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)
